In [274]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [275]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkregressor.csv')

In [276]:
N = 2021

In [277]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [278]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [279]:
print(*data.columns)

season round podium q_delta starting_grid driver_points_before constructor_points_before driver_points_from driver_points_per points_percentage driver_last_3 constructor_last_3 stage_q1 stage_q2 stage_q3


In [280]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        winners += list(prediction_df.predicted_winner.values)
        points += list(prediction_df.predicted_points.values)
        
        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)


        # model.fit(X_train, y_train)

    return points, winners

In [281]:
params = parameters.iloc[0]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                        170
model                    nn_regressor
hidden_layer_sizes    (80, 20, 40, 5)
activation                   logistic
solver                           adam
alpha                        0.042813
score                        0.681818
Name: 0, dtype: object

In [282]:
test = df[(df.season == N)]

model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, 
    alpha=alpha, 
    random_state=1)
    
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [283]:
len(test.query('predicted_winner == 1 & podium == 1'))/ test['round'].nunique()

0.6842105263157895

In [284]:
df = test[['season', 'round', 'podium', 'predicted_winner','starting_grid','driver_points_from', 'predicted_points']]

In [285]:
df.query('predicted_winner == 1').sort_values('predicted_points', ascending=False)

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
2801,2021,4,1,1,1,25.0,9.711259
2896,2021,9,1,1,1,26.0,9.710928
2853,2021,6,15,1,2,0.0,9.710916
2876,2021,8,1,1,1,25.0,9.710866
2821,2021,5,1,1,2,25.0,9.710863
2917,2021,11,2,1,1,18.0,9.710704
2763,2021,2,2,1,1,19.0,9.710699
2858,2021,7,1,1,1,26.0,9.710693
2781,2021,3,1,1,2,25.0,9.710648
3012,2021,17,1,1,1,25.0,9.710621
